<a href="https://colab.research.google.com/github/Kerriea-star/time-series-forecasting-with-lstm-autoencoders/blob/main/Time_Series_Forecasting_with_LSTM_Autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



*   The purpose of this work is to show one way time-series data can be effiently encoded to lower dimensions, to be used into non time-series models.
*   Here I'll encode a time-series of size 12 (12 months) to a single value and use it on a MLP deep learning model, instead of using the time-series on a LSTM model that could be the regular approach.

**Predict future sales**

The task is to forecast the total amount of products sold in every shop for the test set. Note that the list of shops and products slightly changes every month. Creating a robust model that can handle such situations is part of the challenge.

**Data fields description:**

*   ID - an Id that represents a (Shop, Item) tuple within the test set
*   shop_id - unique identifier of a shop
*   item_id - unique identifier of a product
*   item_category_id - unique identifier of item category
*   date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
*   date - date in format dd/mm/yyyy
*   item_cnt_day - number of products sold. You are predicting a monthly amount of this measure
*   item_price - current price of an item
*   item_name - name of item
*   shop_name - name of shop
*   item_category_name - name of item category



In [2]:
# Dependencies
import os, warnings, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import tensorflow as tf
import keras.layers as L

from keras import optimizers, Sequential, Model



In [3]:
# Set seeds to make the experiment more reproducible
def seed_everything(seed=0):
  random.seed(seed)
  np.random.seed(seed)
  tf.random.set_seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  os.environ["TF_DETERMINISTIC_OPS"] = '1'

seed = 0
seed_everything(seed)
warnings.filterwarnings("ignore")
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [6]:
# Loading data
test = pd.read_csv("drive/MyDrive/Colab Notebooks/time-series-forecasting-with-lstm-autoencoders/data/test.csv", dtype={"ID": 'int32', 'shop_id': 'int32', 'item_id': 'int32'})
item_categories = pd.read_csv("drive/MyDrive/Colab Notebooks/time-series-forecasting-with-lstm-autoencoders/data/test.csv", dtype={'item_category': 'str', 'item_category_id': 'int32'})
items = pd.read_csv("drive/MyDrive/Colab Notebooks/time-series-forecasting-with-lstm-autoencoders/data/items.csv", dtype={"item_name": 'str', 'item_id': 'int32'})
shops = pd.read_csv("drive/MyDrive/Colab Notebooks/time-series-forecasting-with-lstm-autoencoders/data/shops.csv", dtype={'shop_name': 'str', 'shop_id': 'int32'})
sales = pd.read_csv("drive/MyDrive/Colab Notebooks/time-series-forecasting-with-lstm-autoencoders/data/sales_train.csv", parse_dates=['date'], dtype={'date': 'str', 'date_block_num': 'int32', 'shop_id': 'int32',
                                                                                                                                                 'item_id': 'int32', 'item_price': 'float32', 'item_cnt_day': 'int32'})
